### Imports 

In [63]:
import requests
import json
from bs4 import BeautifulSoup
import os
from deep_translator import GoogleTranslator
import datetime


### Utils

In [64]:
def translate(text, source ='pl', target ='en'):
    return GoogleTranslator(source, target).translate(text=text)

In [65]:
def extract(ancestor, selector=None, attribute=None, multiple=False):
    if selector:

        if multiple:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:

            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
            
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
        
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()
        

In [66]:
selectors ={
"opinion_id" : (None,"data-entry-id"),
"author" : ("span.user-post__author-name",),            
"recommend" : ("span.user-post_author-recomendation > em",),           
"stars" : ("span.user-post__score-count",),          
"content_pl" : ("div.user-post__text",),
"pros_pl" : ("div.review-feature__item--positive", None, True),
"cons_pl" : ("div.review-feature__item--negative", None, True),
"up_votes" :("button.vote-yes","data-total-vote"),
"down_votes" : ("button.vote-no","data-total-vote"),
"published" : ("span.user-post__published > time:nth-child(1)","datetime"),          
"purchased" : ("span.user-post__published > time:nth-child(2)","datetime")
}

### Extraction of opinions


In [67]:
with open('./cookie.json','r', encoding="UTF-8") as jf:
    headers = json.load(jf)

In [ ]:
product_id = "17230358"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *values)
                for key, values in selectors.items()
            }
            single_opinion["content_en"] = translate(single_opinion["content_pl"])
            single_opinion['pros_en'] = [translate(pros) for pros in single_opinion['pros_pl']]
            single_opinion['cons_en'] = [translate(cons) for cons in single_opinion['cons_pl']]
            single_opinion['recommend'] = True if single_opinion['recommend'] == "Polecam" else False if single_opinion['recommend']== "Nie polecam" else None
            single_opinion['stars'] = float(single_opinion['stars'].split("/")[0].replace(",","."))
            #single_opinion['published'] = datetime.datetime.fromisoformat(single_opinion['published'])
            #single_opinion['purchased'] = datetime.datetime.fromisoformat(single_opinion['purchased']) if single_opinion['purchased'] else None
            single_opinion['up_votes'] = int(single_opinion['up_votes'])
            single_opinion['down_votes'] = int(single_opinion['down_votes'])
            all_opinions.append(single_opinion)

        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None
        
    
    

In [81]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [82]:
with open(f"./opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [83]:
product_id = "17230358"